In [1]:

# 0. Import des librairies

import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eigh
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import time


# 1. Construction de la matrice dynamique
def construire_matrice(NbMol, epsCouple, normaliser=False):
    dim = 2 * NbMol
    M = np.zeros((dim, dim))
    # Diagonale principale
    M[:] = np.identity(dim) * (1.0 + epsCouple)
    M[0, 0] = 1.0
    M[-1, -1] = 1.0

    # Diagonales secondaires
    for j in range(dim - 1):
        c = -1.0 if j % 2 == 0 else -epsCouple
        M[j, j + 1] = c
        M[j + 1, j] = c

    M *= 0.5

    valeurs_propres, vecteurs_propres = eigh(M)

    if normaliser:
        for i in range(vecteurs_propres.shape[1]):
            vecteurs_propres[:, i] /= np.linalg.norm(vecteurs_propres[:, i])

    return M, valeurs_propres, vecteurs_propres


# 2. Diagonalisation complète avec ω/Ω0
def diagonalisation_complete(NbMol, epsCouple, normaliser=False):
    M, vals, vecs = construire_matrice(NbMol, epsCouple, normaliser)
    freqs = np.sqrt(np.abs(vals))
    return M, vals, vecs, freqs


# 3. Question 5(a) – N=10, k'=0
NbMol = 10
epsCouple = 0.0
M, vals, vecs, freqs = diagonalisation_complete(NbMol, epsCouple, normaliser=True)
plt.figure()
plt.plot(freqs, 'o')
plt.title("Fréquences ω/Ω0 pour N=10, k'=0")
plt.xlabel("Index")
plt.ylabel("ω/Ω0")
plt.show()

# 4. Question 5(b) – Temps de diagonalisation
epsCouple = 0.1
Ns = 2 ** np.arange(3, 11)
temps = []
for N in Ns:
    t0 = time.time()
    M, vals, vecs, _ = diagonalisation_complete(N, epsCouple)
    temps.append(time.time() - t0)
plt.figure()
plt.loglog(Ns, temps, 'o-', label="temps mesuré")
plt.loglog(Ns, Ns ** 3 / 1e8, label="loi cubique")
plt.xlabel("N")
plt.ylabel("Temps (s)")
plt.title("Temps de diagonalisation vs N")
plt.legend()
plt.show()

# 5. Question 5(c) – Spectre N=200

NbMol = 200
for eps in [0.1, 1.0]:
    M, vals, vecs, freqs = diagonalisation_complete(NbMol, eps)
    plt.figure()
    plt.plot(freqs, 'o')
    plt.title(f"Spectre ω/Ω0, epsCouple={eps}")
    plt.xlabel("Index")
    plt.ylabel("ω/Ω0")
    plt.show()

# 6. Question 5(d) – Vecteurs propres premiers modes
j_list = [0, 1, 2, 3]
for j in j_list:
    plt.figure()
    plt.plot(vecs[:, j])
    plt.title(f"Vecteur propre j={j + 1}")
    plt.xlabel("Position n")
    plt.ylabel("u_n")
    plt.show()

# 7. Question 5(e) – Modes optiques hauts

j_list = [-1, -2, -3, -4]
for j in j_list:
    plt.figure()
    plt.plot(vecs[:, j])
    plt.title(f"Vecteur propre (haut) index={2 * NbMol + j + 1}")
    plt.xlabel("Position n")
    plt.ylabel("u_n")
    plt.show()

# 8. Question 6 – Branches acoustiques et optiques
epsCouple = 0.1
M, vals, vecs, freqs = diagonalisation_complete(NbMol=200, epsCouple=epsCouple)
N = 200
acoustique = freqs[:N]
optique = freqs[N:][::-1]
plt.figure()
plt.plot(acoustique, label="Acoustique")
plt.plot(range(N, 2 * N), optique, label="Optique")
plt.title("Branches acoustique et optique")
plt.xlabel("Index n")
plt.ylabel("ω/Ω0")
plt.legend()
plt.show()


# 9. Question 7 – Version sparse SciPy

def matrice_sparse(NbMol, epsCouple, k_eigs=20):
    dim = 2 * NbMol
    diag = np.ones(dim) * (1 + epsCouple)
    diag[0] = diag[-1] = 1.0
    offdiag = np.array([-1.0 if j % 2 == 0 else -epsCouple for j in range(dim - 1)])
    M = diags([diag, offdiag, offdiag], [0, 1, -1], format="csr") * 0.5
    vals, vecs = eigsh(M, k=k_eigs, which='SM')
    return M, vals, vecs


# Exemple d'utilisation sparse
M_sparse, vals_sparse, vecs_sparse = matrice_sparse(500, 0.1, k_eigs=10)
print("Valeurs propres calculées sparse :", vals_sparse)


ModuleNotFoundError: No module named 'numpy'